In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,6)

In [3]:
df = pd.read_csv('./data/data_cleaned.csv')

In [4]:
df.head()

,Job_position,Company,Location,Salary,requirements,rating,experience,posting_frequency
0,Big Data Developer,Agile Placement,"Pune, Maharashtra","5,00,000 - 14,00,000 a year",_Strong knowledge of programming and scripting...,0.0,Total work: 6 years,1.0
1,Data Scientist,Client of PlaceElements HR,"Thiruvananthapuram, Kerala","5,00,000 - 10,00,000 a year",Experience in working closely with data analys...,0.0,Python: Algorithms: Statistics: 1 yearData Sci...,1.0
2,Software Engineer - Machine Learning Engineer,Siemens Technology and Services Private Limited,"Pune, Maharashtra",-999,Experience with machine learning architectures...,4.0,NaN,1.0
3,Machine Learning (Fullstack) at Sadhashiva Nag...,Teqlinx Software Solutions LLC,"Bengaluru, Karnataka","30,00,000 a year",• Extremely strong programming background – da...,0.0,NaN,1.0
4,Java/J2EE Developer freshers required,www.yansisofsol.com,"Hyderabad, Telangana","22,000 - 35,000 a month","Sanitising, disinfecting, or cleaning procedur...",0.0,NaN,1.0


In [5]:
# to calculate max and min Salary per annum

yearly_min = {}
yearly_max = {}

def Salary(df):
    
    for i in range(0, len(df)):
        
        if df['Salary'][i] == '-999':
            yearly_min[i] = 0
            yearly_max[i] = 0
            
        if 'a year' in df['Salary'][i]:
            sal_min = df['Salary'][i].split('-')[0].replace('a year','').replace(',','')
            yearly_min[i] = int(sal_min)
            
            try:
                sal_max = df['Salary'][i].split('-')[1].replace('a year','').replace(',','')
                yearly_max[i] = int(sal_max)
                
            # if only single value present will be stored in both max and min, so the average comes accuate
            except:
                sal_max = df['Salary'][i].split('-')[0].replace('a year','').replace(',','')
                yearly_max[i] = int(sal_max)
            
       
        if 'a month' in df['Salary'][i]:
            sal_min = df['Salary'][i].split('-')[0].replace('a month','').replace(',','')
            yearly_min[i] = int(sal_min) * 12
            
            try:
                sal_max = df['Salary'][i].split('-')[1].replace('a month','').replace(',','')
                yearly_max[i] = int(sal_max) * 12    
                
            # if only single value present will be stored in both max and min, so the average comes accuate
            except:
                sal_max = df['Salary'][i].split('-')[0].replace('a month','').replace(',','')
                yearly_max[i] = int(sal_max) * 12
                
        
        if 'an hour' in df['Salary'][i]:
            sal_min = df['Salary'][i].split('-')[0].replace('an hour','').replace(',','')
            yearly_min[i] = int(sal_min) * 9 * 22 * 12
            
            try:
                sal_max = df['Salary'][i].split('-')[1].replace('an hour','').replace(',','')
                yearly_max[i] = int(sal_max) * 9 * 22 * 12  
                
            # if only single value present will be stored in both max and min, so the average comes accuate
            except:
                sal_max = df['Salary'][i].split('-')[0].replace('an hour','').replace(',','')
                yearly_max[i] = int(sal_max) * 9 * 22 * 12
                
Salary(df)

In [6]:
# min, max and avg Salary columns
df['min_Salary'] = pd.DataFrame(yearly_min.values(), index= yearly_min.keys())
df['max_Salary'] = pd.DataFrame(yearly_max.values(), index= yearly_max.keys())
df['avg_yearly_sal'] = ( df['min_Salary'] + df['max_Salary'] )/2
df['monthly_Salary'] = df['avg_yearly_sal']/12.

In [7]:
df['avg_yearly_sal'].fillna(0, inplace=True)
df['min_Salary'].fillna(0, inplace=True)
df['max_Salary'].fillna(0, inplace=True)
df['monthly_Salary'].fillna(0, inplace=True)

Lets just drop these as we got our target column

In [8]:
df.drop('max_Salary', axis=1, inplace=True)
df.drop('min_Salary', axis=1, inplace=True)
df.drop('monthly_Salary', axis=1, inplace=True)

we can divide the annual Salary into 6 differrent categories

In [9]:
df['income_cat'] = pd.cut(df['avg_yearly_sal'], bins=[-999, 0, 50000, 100000, 500000, 1000000, 2500000, np.inf], labels=[-1, 1, 2, 3, 4, 5, 6])

In [10]:
df.drop('Salary', axis=1, inplace=True)

Experience is mentioned in both requirements and experience so we will collect them all and save it in a column of experience 

Some of these requirements mention experienced  

In [11]:
df['experience'] = df['experience'].fillna('na')

In [12]:
net_experience = []
for i in df.experience:
    temp=[]
    for word in i.split():
        if word.isdigit():
            temp.append(word)
    if temp:
        temp.sort(reverse=True)
        net_experience.append(temp[0])
    else:
        net_experience.append(-99)
df['net_experience'] = net_experience

In [13]:
df['net_experience'] = df['net_experience'].astype('int32')

In [14]:
def clean(x):
    for p in ['â', '€', '¦', '“', '¢', '™']:
        x.replace(p, ' ')
    return x
df['requirements'] = df['requirements'].map(clean)

In [15]:
net_experience = []
for i in df.requirements:
    temp=[]
    for word in i.split():
        if word.isdigit():
            temp.append(word)
    if temp:
        temp.sort(reverse=True)
        net_experience.append(temp[0])
    else:
        net_experience.append(-99)
df['exp2'] = net_experience

Removing unwanted values from experience column

In [16]:
for p in ['²', '0080091', '2020', '2024', '2019', '90', '88', '32', '48', '40', '50', '24']:
    df['exp2'] = df['exp2'].apply(lambda x: str(x).replace(p,'-99'))

In [17]:
df['exp2'] = df['exp2'].astype('int32')

where experience required is mentioned in <b>requirements</b> column but missing in <b>experience</b> column 

In [18]:
df['net_experience'] = df['net_experience'].where((df['net_experience']>0), df['exp2'])
df.drop('exp2', axis=1, inplace=True)

In [19]:
df.loc[294, 'experience']

'Python: 1 year'

In [20]:
df.loc[[14, 111, 122, 362, 749], 'requirements']

14     Bachelor’s Degree or higher in data science, S...
111    Exposure to cloud-based machine learning platf...
122    3+ years business experience including analyti...
362    We are looking for an data entry operator And ...
749    Good understanding of server-side CSS preproce...
Name: requirements, dtype: object

Looks like Some openings require no experience

In [21]:
df.loc[[294, 14, 111, 122, 362, 749], 'net_experience'] = 0

Some job positions also mention titles lije junior, intern etc. which require 0 experience, we also want to count that where net experience is missing

In [22]:
for i in df.index:
    if df.loc[i, 'net_experience'] < 0:
        for word in str(df.Job_position[i]).lower().split():
            if word == 'jr' or word == 'junior' or word == 'fresher' or word == 'intern' or word == 'intership' or word == 'interns' or word == 'freshers':
                df.loc[i, 'net_experience'] = 0
            else:
                df[i, 'net_experience'] = -99 

Educational criteria mentioned by these companies can also be useful

In [23]:
def education_level(data):
    if 'bachelor' in data.replace('year',' ').replace("'",' ').lower().split():
        return 'bachelor'
    if 'secondary' in data.replace('year',' ').replace('(',' ').replace("'",' ').lower().split():
        return 'secondary'
    if 'master' in data.replace('year',' ').replace("'",' ').lower().split():
        return 'masters'

In [24]:
df['education_level'] = df['experience'].map(education_level)
df['education_level'].fillna('na',inplace=True)

As the categories of seniority is only jr, senior or na, we can one hot encode them

In [25]:
df = pd.concat([df, pd.get_dummies(df['education_level'])], axis=1)

Seniority of these job positions cal also be useful

In [26]:
def seniority(title):
    title = str(title) 
    if 'ii' in title.lower().split() or 'director' in title.lower().split() or 'specialist' in title.lower().split() or 'professional' in title.lower().split() or 'sr.' in title.lower().split() or 'senior' in title.lower().split():
        return 'senior'
    elif 'i' in title.lower().split() or 'associate' in title.lower().split() or 'junior' in title.lower().split() or 'jr' in title.lower().split()  or 'jr.' in title.lower().split() or 'trainee' in title.lower().split() or 'intern' in title.lower().split() or 'jr.' in title.lower().split():
        return 'jr'
    else:
        return 'na'

In [27]:
# to calculate the seniority of the position applying for
df['job_title'] = df['Job_position'].apply(seniority)

For encoding rank transforamtion, label encoding, frequency encoding were applied but they had very weak correlation with avg_year_Salary <br>
as the categories of seniority is only jr, senior or na, we can one hot encode them

In [28]:
df = pd.concat([df, pd.get_dummies(df['job_title'])], axis=1)

Upon analyzing the requirements column following are the most popular professions <br>
lets store their frequencies

In [29]:
def calc_jobs(data):
    data = data.lower().replace(' ', '')
    if 'machinelearning' in data:
        return 'machine learning'
    
    if 'datascientist' in data:
        return 'data scientist'
    
    if 'softwaredeveloper' in data:
        return 'software developer'
    
    if 'softwareengineer' in data:
        return 'software engineer'
    
    if 'deeplearning' in data:
        return 'deep learning'

In [30]:
df['popular_profession'] = df['requirements'].apply(calc_jobs)


In [31]:
df['popular_profession'] = df['popular_profession'].fillna('na')

In [32]:
df = pd.concat([df, pd.get_dummies(df['popular_profession'])], axis=1)

In [48]:
# we can split the location column and get the state 
df['State'] = df['Location'].apply(lambda x:  x.split(',')[1] if len(x.split(',')) > 1 else x)

We can one hot encode these States values

In [49]:
df = pd.concat([df, pd.get_dummies(df['State'])], axis=1)

In [50]:
# Some companies have multiple job openings this could be useful
df['Company'].value_counts().head()

Larsen & Toubro Infotech Limited    12
Accenture                           10
Angel & Genie                        8
Shaw Academy                         8
Barclays                             7
Name: Company, dtype: int64

In [51]:
job_openings = df['Company'].value_counts()

In [52]:
df['job_openings'] = df['Company'].map(job_openings)

In [53]:
df['requirements'] = df['requirements'].fillna('')

In [54]:
df['job_descr_len'] = df['requirements'].apply(lambda x: 0 if not x else len(x))

lets look at what we have done so far

In [55]:
df.head()

,Job_position,Company,Location,requirements,rating,experience,posting_frequency,avg_yearly_sal,income_cat,net_experience,...,Jharkhand,Karnataka,Kerala,Maharashtra,Meghalaya,Punjab,Remote,Tamil Nadu,job_openings,job_descr_len
0,Big Data Developer,Agile Placement,"Pune, Maharashtra",_Strong knowledge of programming and scripting...,0.0,Total work: 6 years,1.0,950000.0,4,6,...,0,0,0,0,0,0,0,0,1,158
1,Data Scientist,Client of PlaceElements HR,"Thiruvananthapuram, Kerala",Experience in working closely with data analys...,0.0,Python: Algorithms: Statistics: 1 yearData Sci...,1.0,750000.0,4,2,...,0,0,0,0,0,0,0,0,1,156
2,Software Engineer - Machine Learning Engineer,Siemens Technology and Services Private Limited,"Pune, Maharashtra",Experience with machine learning architectures...,4.0,na,1.0,0.0,-1,-99,...,0,0,0,0,0,0,0,0,7,149
3,Machine Learning (Fullstack) at Sadhashiva Nag...,Teqlinx Software Solutions LLC,"Bengaluru, Karnataka",• Extremely strong programming background – da...,0.0,na,1.0,3000000.0,6,-99,...,0,0,0,0,0,0,0,0,1,160
4,Java/J2EE Developer freshers required,www.yansisofsol.com,"Hyderabad, Telangana","Sanitising, disinfecting, or cleaning procedur...",0.0,na,1.0,342000.0,3,0,...,0,0,0,0,0,0,0,0,1,160


### Analyzing Job skills

As due to covid-19 many people working in the industry have lost their jobs, and according to news articles the skill demand for job industry 
is also changing, lets take a look at the skills, in demand in the job industry 

In [56]:
from nltk.tokenize import word_tokenize

In [57]:
requirements = df['requirements']

In [58]:
requirements = list(filter(None, requirements))

In [59]:
# split punctuation 
for p in ['-','(',')','.','/']:
    job_descr = []
    for i in range(0, len(requirements)):
        c = requirements[i].split(p)
        for x in c:
            x.replace('.',' ')
            job_descr.append(x)

In [60]:
# Remove punctuation and convert to lower case
for x in range(0,len(job_descr)):
    for p in ['.', '-', ')', '(', '…', ',', ':', "'"]:
        job_descr[x] = job_descr[x].replace(p,' ')
    job_descr[x] = job_descr[x].lower()       
    

In [63]:
# analyzing keywords from custom keyword list
f = open("./utils/skills.txt","r",) 
skills=[]
for x in f:
    skills.append(x)
f.close()

In [64]:
for i in skills:
    skills = i.split(',')

In [65]:
for i in range(0, len(skills)):
    skills[i] = skills[i].replace(' ','')
    skills[i] = skills[i].lower()

In [66]:
print(skills)

['html', 'css', 'c', 'debugging', 'git', 'graphic', 'database', 'java', 'javascript', 'nodejs', 'apis', 'r', 'ruby', 'php', 'net', 'c#', 'jquery', 'python', 'perl', 'react', 'reactjs', 'ux', 'ui', 'testing', 'analyst', 'springboot', 'jpa', 'microservice', 'postman', 'rest', 'api', 'angular', 'azure', 'aws', 'cloud', 'debugging', 'btech', 'cs', 'idbc', 'porting', 'porting', 'vive', 'go', 'playstation', 'medicine', 'food', 'chromium', 'nodejs', 'js', 'net', 'server', 'architecture', 'mobile', 'b2b', 'healthcare', 'security', 'sql', 'mvc', 'asp', 'version', 'typescript', 'sass', 'pwas', 'ios', 'androis', 'html', 'python.node', 'rdbms', 'mysql', 'script', 'query', 'mongo', 'oops', 'os', 'restful', 'app', 'xamarin', 'financial', 'shell', 'unix', 'script', 'powershell', 'linux', 'game', 'gui', 'unity', 'ai', 'jquery', 'iot', 'freelance', 'bootstrap', 'word', 'excel', 'swift', 'salesforce', 'graphic', 'github', 'flutter', 'c++', 'c#', 'docker', 'stack', 'bug', 'lravel', 'flux', 'redux', 'nlp'

Processing requirements on the basis whether a skill is present in that row

In [67]:
df['requirements'] = df['requirements'].apply(lambda x: ' '.join([word for word in x.lower().split() if word in (skills)]))
df['requirements'].replace(to_replace='', value='na', inplace=True)

c:\python\python38\lib\site-packages\pandas\core\series.py:4479: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [68]:
processed_text = word_tokenize(str(job_descr))

In [69]:
# to calculate the frequency of a particular skill mentioned in job description
def calc_skill_freq(data):
    skill_dict = {}

    for i in range(1,len(data)):
        token = data[i]
        if token in skills:
            try:
                skill_dict[token].add(i)
            except:
                skill_dict[token] = {i}
            
    for i in skill_dict:
        skill_dict[i] = len(skill_dict[i])
        
    return skill_dict

In [70]:
job_descr_dict = calc_skill_freq(processed_text)

Some Companies have mentioned the required skills in job position and some in some description <br>
Lets take a look at the skills mentioned in Job description column, then we will add them to get skills in demand

In [71]:
# remove punctuation present in job position column
def remove_punctuation(df):
    for p in ['/', ',', '(', ')', '-', '|', '&', '_', '.', '“', '”', ':']:
        df['Job_position'] = df['Job_position'].apply(lambda x: str(x).replace(p,' '))

    return df

In [72]:
df = remove_punctuation(df)

In [74]:
# analyzing stopwords from custom stopwords list
f = open("./utils/stopwords.txt","r",) 
stopwords=[]

for x in f:
    stopwords.append(x)
    
f.close()

In [75]:
for i in stopwords:
    stopwords = i.split(',')

In [76]:
for i in range(0, len(stopwords)):
    stopwords[i] = stopwords[i].replace("'","")
    stopwords[i] = stopwords[i].replace(" ","")
    stopwords[i] = stopwords[i].lower()

In [77]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [78]:
# removing stopwords from the Job_position column
job_role = list(df['Job_position'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in (stopwords)])))

In [79]:
df['Job_position'] = df['Job_position'].apply(lambda x: ' '.join([word for word in x.lower().split() if word in (skills)]))
df['Job_position'] = df['Job_position'].where(df['Job_position'] != '', 'na')

In [80]:
job_role = word_tokenize(str(job_role))

In [81]:
# Now calculate the frequency of a particular skill mentioned in job role
job_role_dict = calc_skill_freq(job_role)

Below we first pass all the elements of the first dictionary into the third one and then pass the second dictionary 
into the third. This will replace the duplicate keys of the first dictionary. <br>
More info : (https://www.geeksforgeeks.org/python-merging-two-dictionaries/)

In [82]:
skills_dict = {**job_role_dict, **job_descr_dict}

Lets save this dictionary for now it will be useful for EDA

In [84]:
#import pickle
#skill_file = open('./utils/skill_dictionary', 'wb') 
#pickle.dump(skills_dict, skill_file) 
#skill_file.close() 

In [85]:
# now create new column for each skill with value equal to the frequency of that skill occurring in that particular cell
def calc_freq(df):
    for key in list(skills_dict.keys()):
        if skills_dict[key] > 15:
            skill_calc = []
            for i in range(0,len(df)):
                count = 0
                
                # here we are counting frquency from both requirements and Job position column
                for word in df['requirements'][i].lower().split() :
                    if key in df['Job_position'][i].lower().split():
                        count += 1
                    if key == word:
                        count += 1
                        skill_calc.append(count)
                    else:
                        skill_calc.append(0)
                        
            df = pd.concat([df, pd.DataFrame(skill_calc, columns=[key])], axis=1)
            # all the missing values should be filled with zero as they dont contain that particular skill 
            df[key] = df[key].fillna(0)
    return df
df = calc_freq(df)

As skills from job position and description were added its possible some of them dont appear in description, their frequency wiil be zero so we 
must drop them

In [86]:
# remove columns with constant values
df = df.loc[:, (df != df.iloc[0]).any()] 

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737 entries, 0 to 1736
Columns: 109 entries, Job_position to apis
dtypes: category(1), float64(42), int32(1), int64(2), object(9), uint8(54)
memory usage: 819.8+ KB


In [88]:
#df.to_csv('./data/data_prepared.csv', index=False)